入出力画像の一致度で画像選別

In [ ]:
# CIVE = 0.441 * R - 0.811 * G + 0.385 * B + 18.78745
# cive > 0なら緑
# civeとExGの共通範囲狭いかも

import os
import cv2
import numpy as np
import json

def calculate_ExG_ratio(image, mask):
    b, g, r = cv2.split(image)
    cive = 0.441 * r - 0.811 * g + 0.385 * b + 18.78745
    ExG = 2 * g - r - b
    condition = (r + g + b > 0) & ((cive > 0) | (ExG < 0))
    masked_condition = condition & (mask > 0)
    ratio = np.sum(masked_condition) / np.sum(mask > 0)
    #print(ratio)
    return ratio

def process_images(line2line_path, gendata_path):
    result = {}
    for filename in os.listdir(line2line_path):
        if filename.endswith('.jpg'):
            line2line_img_path = os.path.join(line2line_path, filename)
            gendata_img_path = os.path.join(gendata_path, filename)
            
            if not os.path.exists(gendata_img_path):
                print(f"Warning: {filename} not found in gendata path")
                continue
            
            line2line_img = cv2.imread(line2line_img_path, 0)
            gendata_img = cv2.imread(gendata_img_path)
            
            if line2line_img is None or gendata_img is None:
                print(f"Warning: Could not read image {filename}")
                continue
            
            mask = np.uint8(line2line_img > 128)
            
            if np.sum(mask) == 0:
                print(f"Warning: No white area in mask for {filename}")
                continue
            
            ratio = calculate_ExG_ratio(gendata_img, mask)
            result[filename] = ratio
    
    return result

def get_top_100_images(result):
    sorted_result = sorted(result.items(), key=lambda x: x[1], reverse=True)
    return [filename for filename, _ in sorted_result[:100]]
    # return [filename for filename, _  in sorted_result[-100:]] # 逆順
    #50-100
    #return [filename for filename, _ in sorted_result[50:100]]
    # return [filename for filename, _ in sorted_result[-100:-50]]


def get_image_ids(json_file, top_100_images):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    image_id_map = {img['file_name']: img['id'] for img in data['images']}
    return [image_id_map[img] for img in top_100_images if img in image_id_map]

def main():
    num_list = ["01", "02", "03", "04", "05"]
    for num in num_list:
        line2line_path = f"../images/no_extend/{num}/line2line/"
        gendata_path = f"../images/gen_data/output/0718_best/multi_controlnet/{num}/"
        # line2line_path = f"../images/test2/{num}/line2line/"
        # gendata_path = f"../images/gen_data/output/1209/leaf_block_mcn/{num}/"
        gendata_json = f"../images/gen_data/{num}_gen_coco.json"
        
        result = process_images(line2line_path, gendata_path)
        
        top_100_images = get_top_100_images(result)
        image_ids = get_image_ids(gendata_json, top_100_images)
        
        output_file = f"../ids/select_ExG/top_100_{num}.txt"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        with open(output_file, 'w') as f:
            f.write(','.join(map(str, image_ids)))
        
        print(f"{num} Done")
if __name__ == "__main__":
    main()

1ペアに対してExG<0&&CIVE<0の領域抽出

In [ ]:
import cv2
import numpy as np

def calculate_ExG_CIVE(image):
    b, g, r = cv2.split(image)
    b = b.astype(float)
    g = g.astype(float)
    r = r.astype(float)
    
    cive = 0.441 * r - 0.811 * g + 0.385 * b + 18.78745
    ExG = 2 * g - r - b
    
    return ExG, cive

def process_image_pair(line2line_path, gendata_path, output_path):
    # 線画（マスク）画像を読み込む
    line2line_img = cv2.imread(line2line_path, 0)
    
    # 生成データ（カラー）画像を読み込む
    gendata_img = cv2.imread(gendata_path)
    
    if line2line_img is None or gendata_img is None:
        print("エラー: 画像を読み込めませんでした。")
        return
    
    # マスクを作成（白い領域を抽出）
    mask = np.uint8(line2line_img > 128)
    
    # ExGとCIVEを計算
    ExG, cive = calculate_ExG_CIVE(gendata_img)
    result = np.zeros_like(gendata_img)
    condition = (mask > 0) & (cive > 0)
    masked_condition = condition & (mask > 0)
    ratio = np.sum(masked_condition) / np.sum(mask > 0)
    result[condition] = [255, 255, 255]
    print(ratio)
    
    
    # 結果を保存
    cv2.imwrite(output_path, result)
    
    print(f"処理完了: 結果を {output_path} に保存しました。")

# 使用例
bw_image_path = '../images/test2/01/line2line/01_68_generate_image.jpg'
#color_image_path = '../images/gen_data/output/0718_best/multi_controlnet/01/01_0_generate_image.jpg'
color_image_path = '../images/gen_data/output/1209/leaf_block_mcn/01/01_68_generate_image.jpg'
output_path = './output.jpg'
process_image_pair(bw_image_path, color_image_path, output_path)



横軸：割合，縦軸：枚数のグラフ

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def calculate_ExG_ratio(image, mask):
    b, g, r = cv2.split(image)
    cive = 0.441 * r - 0.811 * g + 0.385 * b + 18.78745
    ExG = 2 * g - r - b
    condition = (r + g + b > 0) & ((cive > 0) | (ExG < 0))
    masked_condition = condition & (mask > 0)
    ratio = np.sum(masked_condition) / np.sum(mask > 0)
    return ratio

def process_images(line2line_path, gendata_path):
    result = {}
    for filename in os.listdir(line2line_path):
        if filename.endswith('.jpg'):
            line2line_img_path = os.path.join(line2line_path, filename)
            gendata_img_path = os.path.join(gendata_path, filename)
            
            if not os.path.exists(gendata_img_path):
                print(f"Warning: {filename} not found in gendata path")
                continue
            
            line2line_img = cv2.imread(line2line_img_path, 0)
            gendata_img = cv2.imread(gendata_img_path)
            
            if line2line_img is None or gendata_img is None:
                print(f"Warning: Could not read image {filename}")
                continue
            
            mask = np.uint8(line2line_img > 128)
            
            if np.sum(mask) == 0:
                print(f"Warning: No white area in mask for {filename}")
                continue
            
            ratio = calculate_ExG_ratio(gendata_img, mask)
            result[filename] = ratio
    
    return result

def plot_line_and_histogram(all_ratios_list, labels):
    plt.figure(figsize=(12, 8))
    
    bin_edges = np.arange(0, 1.1, 0.1)
    colors = ['blue', 'red']
    
    for ratios, label in zip(all_ratios_list, labels):
        hist, _ = np.histogram(ratios, bins=bin_edges)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        # Plot histogram alpha=で透過率をいじる
        plt.bar(bin_centers, hist, width=0.08, color=colors, label=label)
        
        # Plot line graph
        plt.plot(bin_centers, hist, marker='o', label=label, linewidth=6, markersize=15)
    
    plt.xlabel('Ratio', fontsize=20)
    plt.ylabel('Number of Images', fontsize=20)
    # plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    # plt.title('Distribution of Image Ratios', y=-0.12)
    plt.grid(True, alpha=0.3)
    
    plt.xlim(0, 1)
    plt.ylim(0, plt.ylim()[1])

    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.tick_params(axis='both', which='major', labelsize=20)
    
    plt.gca().set_aspect('auto', adjustable='box')
    
    plt.show()

def main():
    num_list = ["01", "02", "03", "04", "05"]
    gendata_paths = [
    ("../images/gen_data/output/0718_best/multi_controlnet/", " "),
    ("../images/gen_data/output/0718_best/controlnet/", " ")
    ]

    all_ratios_list = []
    labels = []

    for gendata_path, label in gendata_paths:
        all_ratios = []

        for num in num_list:
            line2line_path = f"../images/no_extend/{num}/line2line/"
            current_gendata_path = os.path.join(gendata_path, num)
            
            result = process_images(line2line_path, current_gendata_path)
            all_ratios.extend(result.values())
            
            print(f"{num} Done")

        all_ratios_list.append(all_ratios)
        labels.append(label)

    # Plot line graphs and histograms for all ratios
    labels = ["ours", "prior"]
    plot_line_and_histogram(all_ratios_list, labels)

if __name__ == "__main__":
    main()

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def calculate_ExG_ratio(image, mask):
    b, g, r = cv2.split(image)
    cive = 0.441 * r - 0.811 * g + 0.385 * b + 18.78745
    ExG = 2 * g - r - b
    condition = (r + g + b > 0) & ((cive > 0) | (ExG < 0))
    masked_condition = condition & (mask > 0)
    ratio = np.sum(masked_condition) / np.sum(mask > 0)
    return ratio

def process_images(line2line_path, gendata_path):
    result = {}
    for filename in os.listdir(line2line_path):
        if filename.endswith('.jpg'):
            line2line_img_path = os.path.join(line2line_path, filename)
            gendata_img_path = os.path.join(gendata_path, filename)
            
            if not os.path.exists(gendata_img_path):
                print(f"Warning: {filename} not found in gendata path")
                continue
            
            line2line_img = cv2.imread(line2line_img_path, 0)
            gendata_img = cv2.imread(gendata_img_path)
            
            if line2line_img is None or gendata_img is None:
                print(f"Warning: Could not read image {filename}")
                continue
            
            mask = np.uint8(line2line_img > 128)
            
            if np.sum(mask) == 0:
                print(f"Warning: No white area in mask for {filename}")
                continue
            
            ratio = calculate_ExG_ratio(gendata_img, mask)
            result[filename] = ratio
    
    return result

def plot_line_and_histogram(all_ratios_list, labels):
    plt.figure(figsize=(12, 8))
    
    bin_edges = np.arange(0, 1.1, 0.1)
    colors = ['#CC6677', '#4477AA']  # 論文に適した色を設定
    
    # ヒストグラムを重ねて表示するため、透明度を調整
    alphas = [0.5, 0.5]
    bar_width = 0.09
    
    for i, (ratios, label) in enumerate(zip(all_ratios_list, labels)):
        hist, _ = np.histogram(ratios, bins=bin_edges)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        # ヒストグラム描画（重ね合わせ）
        plt.bar(bin_centers, hist, width=bar_width, alpha=alphas[i], 
                color=colors[i], edgecolor=colors[i], linewidth=1.5, label=f"{label} (Hist)")
        
        # 同じデータに対して折れ線グラフを描画
        plt.plot(bin_centers, hist, marker='o', color=colors[i], 
                linewidth=3, markersize=10, label=f"{label} (Line)")
    
    plt.xlabel('Ratio', fontsize=20)
    plt.ylabel('Num Images', fontsize=20)
    plt.legend(fontsize=20)
    plt.grid(True, alpha=0.3)
    
    plt.xlim(0, 1)
    plt.ylim(0, plt.ylim()[1] * 1.1)  # 上限に少し余裕を持たせる

    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.tick_params(axis='both', which='major', labelsize=20)
    
    plt.tight_layout()
    
    # 論文用に高品質な画像として保存
    plt.savefig('ratio_comparison.pdf', format='pdf', bbox_inches='tight', dpi=300)
    plt.savefig('ratio_comparison.png', format='png', bbox_inches='tight', dpi=300)
    
    plt.show()

def main():
    num_list = ["01", "02", "03", "04", "05"]
    gendata_paths = [
    ("../images/gen_data/output/0718_best/multi_controlnet/", "Multi ControlNet"),
    ("../images/gen_data/output/0718_best/controlnet/", "ControlNet")
    ]

    all_ratios_list = []
    labels = []

    for gendata_path, label in gendata_paths:
        all_ratios = []

        for num in num_list:
            line2line_path = f"../images/no_extend/{num}/line2line/"
            current_gendata_path = os.path.join(gendata_path, num)
            
            result = process_images(line2line_path, current_gendata_path)
            all_ratios.extend(result.values())
            
            print(f"{num} Done")

        all_ratios_list.append(all_ratios)
        labels.append(label)

    # 重ね合わせヒストグラムと折れ線グラフをプロット
    labels = ["ours", "prior"]
    plot_line_and_histogram(all_ratios_list, labels)

if __name__ == "__main__":
    main()